In [1]:
import json
import os
from datetime import datetime, timedelta, timezone
from pymongo import MongoClient
from dotenv import load_dotenv
import uuid

load_dotenv()

# --- prior inventories doc (yesterday) ---
previous_docs = [
    {
        "id": "9f3a27e6-1b52-4e0b-bc6e-62c1f2a7c8b9",
        "previous_form_qr_code": "20251011",
        "current_form_qr_code": "20251012",
        "date": "2025-10-12",
        "location": "JEF Gas Station – Sikatuna Branch",
        "tin": "123-456-789",
        "branch": "JEF Biosciences Fuel Division",
        "items": [
            {"item_number": "1", "barcode": "4809992000101", "type": "fuel", "name": "Premium Gasoline", "price": 69.5, "unit": "liter", "previous_quantity": 520, "addstock": 300, "sold": 250, "current_quantity": 570},
            {"item_number": "2", "barcode": "4809992000102", "type": "fuel", "name": "Regular Gasoline", "price": 64.75, "unit": "liter", "previous_quantity": 680, "addstock": 400, "sold": 320, "current_quantity": 760},
            {"item_number": "3", "barcode": "4809992000103", "type": "fuel", "name": "Premium Diesel", "price": 61.2, "unit": "liter", "previous_quantity": 450, "addstock": 350, "sold": 280, "current_quantity": 520},
            {"item_number": "4", "barcode": "4809992000201", "type": "lubricant", "name": "Engine Oil SAE 40", "price": 250.0, "unit": "liter", "previous_quantity": 60, "addstock": 40, "sold": 25, "current_quantity": 75},
            {"item_number": "5", "barcode": "4809992000202", "type": "lubricant", "name": "Automatic Transmission Fluid (ATF)", "price": 310.0, "unit": "liter", "previous_quantity": 50, "addstock": 30, "sold": 20, "current_quantity": 60},
            {"item_number": "6", "barcode": "4809992000203", "type": "lubricant", "name": "Gear Oil EP 90", "price": 285.0, "unit": "liter", "previous_quantity": 45, "addstock": 25, "sold": 15, "current_quantity": 55},
            {"item_number": "7", "barcode": "4809992000204", "type": "lubricant", "name": "Hydraulic Oil ISO 68", "price": 295.0, "unit": "liter", "previous_quantity": 35, "addstock": 25, "sold": 10, "current_quantity": 50},
            {"item_number": "8", "barcode": "4809992000205", "type": "lubricant", "name": "2T Motorcycle Oil", "price": 180.0, "unit": "liter", "previous_quantity": 80, "addstock": 60, "sold": 45, "current_quantity": 95}
        ]
    }
]

prev = previous_docs[0]
prev_date = datetime.strptime(prev["date"], "%Y-%m-%d")
new_date = (prev_date + timedelta(days=1)).strftime("%Y-%m-%d")
now = datetime.now(timezone.utc)

def make_item(it):
    return {
        "item_number": it["item_number"],  # copy same line number
        "id": str(uuid.uuid4()),
        "created": now,
        "updated": now,
        "barcode": it["barcode"],
        "type": it["type"],
        "name": it["name"],
        "price": it["price"],
        "unit": it["unit"],
        "previous_quantity": it["current_quantity"],
        "addstock": 0,
        "sold": 0,
        "current_quantity": 0
    }

new_doc = {
    "id": str(uuid.uuid4()),
    "previous_form_qr_code": prev["current_form_qr_code"],
    "current_form_qr_code": "87654321",
    "date": new_date,
    "location": prev["location"],
    "tin": prev["tin"],
    "branch": prev["branch"],
    "created": now,
    "items": [make_item(it) for it in prev["items"]],
    "is_empty": True
}

# --- insert to MongoDB ---
uri = os.getenv("MONGODB_URI")
database = os.getenv("MONGODB_DATABASE")
collection_name = "inventories"

if uri and database:
    client = MongoClient(uri)
    coll = client[database][collection_name]
    result = coll.insert_one(dict(new_doc))
    print(f"inserted_id: {str(result.inserted_id)}")

# Pretty print
print(json.dumps(new_doc, ensure_ascii=False, indent=2))


inserted_id: 68ee73d2c9cd8d0decc4ccbb


TypeError: Object of type datetime is not JSON serializable